<a href="https://colab.research.google.com/github/deiveleal/data/blob/main/mestrado/ft105/classificacao/MultilayerPerceptronsClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#FT105A - Tópico Interdisciplinar I: Introdução ao Aprendizado de Máquina
### Aluno: Deive Audieres Leal
### RA: 083423


### Tarefa 3: Classification with Multilayer Perceptrons (MLP)

#### Link para o repositório: [MultilayerPerceptronsClassifier](https://github.com/deiveleal/data/blob/main/mestrado/ft105/classificacao/MultilayerPerceptrons-MLP.ipynbb)

#### Instala o repositório da UCI

In [1]:
!pip install ucimlrepo

#### Importação das bibliotecas

In [2]:
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from ucimlrepo import fetch_ucirepo
from scipy import stats
import numpy as np

#### Importa o conjunto de dados

#### Realiza a busca dos dados


In [3]:
liver_disorders = fetch_ucirepo(id=60)

#### Os dados são um dataframe pandas já separado em features e targets (atributo alvo)

In [4]:
X = liver_disorders.data.features
y = liver_disorders.data.targets.astype(str)

### Mostra o dataframe montado

In [5]:
pd.concat([X,y], axis=1).head(7)

,mcv,alkphos,sgpt,sgot,gammagt,drinks
0,85,92,45,27,31,0.0
1,85,64,59,32,23,0.0
2,86,54,33,16,54,0.0
3,91,78,34,24,36,0.0
4,87,70,12,28,10,0.0
5,98,55,13,17,17,0.0
6,88,62,20,17,9,0.5


##### Cria função com o objetivo de padronizar os valores de entrada do treinamento para maximizar o desempenho. Assim, todas as variáveis de treinamento terão um valor aproximado.

In [6]:
def standard_scaler_values(X_train, X_test):
    sc=StandardScaler()
    scaler = sc.fit(X_train)
    trainX_scaled = scaler.transform(X_train)
    testX_scaled = scaler.transform(X_test)
    return trainX_scaled, testX_scaled

##### Cria função com o modelo de MLP. Foram usados os parâmetros: 25 camadas, a ativação foi logistica, função sigmóide, o peso foi configurado para ser do tipo gradiente estocástico e com um máximo de 300 iterações.

In [7]:
def mlp_classifier(X_train, X_test, y_train, y_test):
    clf = MLPClassifier(hidden_layer_sizes=25, solver='sgd', activation="logistic", max_iter=300)
    clf = clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    return accuracy, y_pred

#### Treina os classificadores e realiza a previsão com 5 rodadas de treino e classificação

In [8]:
results = {}
for execution in range(0, 5):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    X_train_s, X_test_s = standard_scaler_values(X_train=X_train, X_test=X_test)
    results[f"mlp_accuracy_{execution}"], results[f"mlp_classification_{execution}"] = mlp_classifier(X_train_s, X_test_s, y_train, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1098: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1098: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't conver

#### Cria dataframe com os resultados da classificação

In [9]:
df_results = pd.DataFrame(results)
df_results

,mlp_accuracy_0,mlp_classification_0,mlp_accuracy_1,mlp_classification_1,mlp_accuracy_2,mlp_classification_2,mlp_accuracy_3,mlp_classification_3,mlp_accuracy_4,mlp_classification_4
0,0.375,0.5,0.307692,0.5,0.278846,0.5,0.307692,0.5,0.317308,0.5
1,0.375,0.5,0.307692,0.5,0.278846,0.5,0.307692,0.5,0.317308,0.5
2,0.375,0.5,0.307692,0.5,0.278846,0.5,0.307692,0.5,0.317308,0.5
3,0.375,0.5,0.307692,0.5,0.278846,0.5,0.307692,0.5,0.317308,0.5
4,0.375,0.5,0.307692,0.5,0.278846,0.5,0.307692,0.5,0.317308,0.5
...,...,...,...,...,...,...,...,...,...,...
99,0.375,0.5,0.307692,0.5,0.278846,0.5,0.307692,0.5,0.317308,0.5
100,0.375,0.5,0.307692,0.5,0.278846,0.5,0.307692,0.5,0.317308,0.5
101,0.375,0.5,0.307692,0.5,0.278846,0.5,0.307692,0.5,0.317308,0.5
102,0.375,0.5,0.307692,0.5,0.278846,0.5,0.307692,0.5,0.317308,0.5


In [10]:
print("Acurácia média:", sum([df_results.mlp_accuracy_0[0], df_results.mlp_accuracy_1[0], df_results.mlp_accuracy_2[0], df_results.mlp_accuracy_3[0], df_results.mlp_accuracy_4[0]])/5)

Acurácia média: 0.3173076923076923


Em comparação com os outros classificadores, mesmo quando utilizado a técnica do ensemble, os valores de acurácia eram no geral menor que 20. Utilizando MLP, o valor da acurácia subiu para mais de 30, sendo que chegou a bater os 40 em algumas testagens. Ao menos com esse conjunto de testes, MLP demonstra superioridade à outras técnicas.